# Test #1 - The influence of geology, faults and fractures

## Initialization

In [1]:
# Loads pyKasso and other useful packages
import copy
import numpy as np
import pyvista as pv
import pykasso as pk
import pykasso.visualization as pkv

pv.global_theme.notebook = True

In [2]:
# Creates a new project directory
project_path = 'geometry_01/'
pk.create_project(project_path)

In [3]:
# Constructs the settings
sks_settings = {
    'sks' : {
        'seed' : 1111,
    },
    'grid' : {
        'x0' : 0,
        'y0' : 0,
        'z0' : 100,
        'nx' : 100,
        'ny' : 50,
        'nz' : 50,
        'dx' : 10,
        'dy' : 10,
        'dz' : 10
    },
    'outlets' : {
        'number'     : 1,
        'data'       : [[994, 250]],
        'importance' : [1],
        'mode'       : 'domain_bottom',
    },
    'inlets' : {
        'number'     : 1,
        'data'       : [[0, 250]],
        'per_outlet' : [1],
        'importance' : [1],
        'mode'       : 'domain_surface',
    },
}

## Easiest case : No domain delimitations + no geological features 

In [4]:
# Computes the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    sks_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=sks_settings)
    sim.build_model()
    # sim.compute_karst_network()

In [5]:
help(sim.domain._get_subdomain)

Help on method _get_subdomain in module pykasso.core.domain:

_get_subdomain(subdomain_name: str) -> numpy.ndarray method of pykasso.core.domain.Domain instance
    Returns the numpy.ndarray modeling the requested subdomain.
    
    Parameters
    ----------
    subdomain_name : str
        Name of the requested subdomain: 
        - "domain"
        - "domain_surface"
        - "domain_bottom"
        - "domain_borders"
        - "domain_borders_sides"
        - "domain_borders_surface"
        - "domain_borders_bottom"
        - "vadose"
        - "vadose_borders"
        - "phreatic"
        - "phreatic_surface"
        - "phreatic_borders_surface"
    
        
    Returns
    -------
    out : np.ndarray



In [5]:
# Visualizes the results
visualization = pkv.Visualizer(project_path)
settings = {
    'ghosts'  : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
}
visualization.show([0,1], ['karst'], settings=settings)

dict_keys(['maps', 'vectors', 'inlets', 'outlets', 'grid', 'domain', 'geology', 'faults', 'fractures'])


KeyError: 'karst'

## Influence of the faults

In [6]:
# Constructs the faults
faults = np.full((100,50,50), 0)
faults[:, :, 25] = 1

# Visualizes the faults
pkv.show_array(faults)

# Updates the settings
new_settings = copy.deepcopy(sks_settings)
new_settings['faults'] = {'data' : faults}

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [7]:
# Computes the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    new_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=new_settings)
    sim.build_model()
    sim.compute_karst_network()

In [8]:
# Visualizes the results
visualization = pkv.Visualization(path_project)
settings = {
    'ghosts'  : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
}
visualization.show([2, 3], ['karst', 'faults'], settings=settings)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Influence of the fracturation

In [9]:
# Declares the fracturation settings
fractures = {
    'seed': 111111,
    'settings': {
        'family_01': {
            'alpha'        : 2,
            'density'      : 0.00005,
            'orientation'  : 180,
            'dip'          : [10,20],
            'length'       : 500,
        }, 
        'family_02': {
            'alpha'        : 2,
            'density'      : 0.00005,
            'orientation'  : [340,20],
            'dip'          : 90,
            'length'       : [400,600],
        }
    }
}

# Updates the settings
new_settings = copy.deepcopy(sks_settings)
new_settings['fractures'] = fractures

In [10]:
# Computes the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    new_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=new_settings)
    sim.build_model()
    sim.compute_karst_network()

In [11]:
sim.fractures.table

,family_id,x,y,z,radius,orientation,dip,normal
0,1,1085.616879,699.204133,808.114736,250.000000,10313.240312,0.320028,"(-0.5984507336140548, 0.8011401384666106, -0.9..."
1,1,580.889012,332.543086,564.839089,250.000000,10313.240312,0.244019,"(-0.5984546414671323, 0.8011453698712689, -0.9..."
2,1,356.332719,532.351925,796.243942,250.000000,10313.240312,0.326921,"(-0.5984503271325389, 0.8011395943137845, -0.9..."
3,1,172.467134,-22.855182,-53.865691,250.000000,10313.240312,0.251702,"(-0.5984542943263113, 0.8011449051572501, -0.9..."
4,1,1028.131196,531.745430,-154.287175,250.000000,10313.240312,0.212136,"(-0.5984559671225481, 0.8011471445131964, -0.9..."
...,...,...,...,...,...,...,...,...
85,2,366.162788,573.618073,503.503779,209.476981,-7.545726,90.000000,"(6.07020909366049e-17, 8.040871381808922e-18, ..."
86,2,251.024767,-214.198563,635.645538,257.162489,-10.087476,90.000000,"(6.028577922314361e-17, 1.0724937301123499e-17..."
87,2,-280.825382,-154.826761,669.934158,250.370216,2.084697,90.000000,"(6.119181304315131e-17, -2.227436451767254e-18..."
88,2,718.452033,-137.800788,-149.804165,209.293433,7.135270,90.000000,"(6.075813605578886e-17, -7.605810915404993e-18..."


In [12]:
# Visualizes the results
visualization = pkv.Visualization(path_project)
settings = {
    'ghosts'  : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
}
visualization.show([4,5], ['karst', 'fractures'], settings=settings)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

## Influence of the geology

In [13]:
# Constructs the geology
geology = np.full((100,50,50), 1)
geology[:, 35:50, :] = 2

# Visualizes the geology
pkv.show_array(geology)

# Updates the settings
new_settings = copy.deepcopy(sks_settings)
new_settings['geology'] = {
    'data' : geology,
    'costs' : {1: 0.6, 2: 0.4}
}

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [14]:
# Computes the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    new_settings['sks']['algorithm'] = algorithm
    sim = pk.SKS(sks_settings=new_settings)
    sim.build_model()
    sim.compute_karst_network()

In [15]:
# Visualizes the results
visualization = pkv.Visualization(path_project)
settings = {
    'ghosts'  : [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'cpos'    : 'xz',
}
visualization.show([6,7], ['karst', 'cost'], settings=settings)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)